# 00 - Importação de Bilbiotecas


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pandas as pd
import os 
import zipfile
import glob

spark = SparkSession.builder.getOrCreate()

# 01 - Etapa de Descompactação dos Arquivos

In [3]:
unzip_file_path = fr"C:\Temp\Iesus\dados\arquivo_csv"

zip_file_path =  fr"C:\Temp\Iesus\dados\arquivo_zip"

file_list = os.listdir(zip_file_path)

for file in file_list:
    
    if file.endswith('.zip'):
        
        x = zip_file_path + '\\' + file
        print (x)
        zip = zipfile.ZipFile(x)
        zip.extractall(unzip_file_path)
        zip.close()
        os.remove(x)

C:\Temp\Iesus\dados\arquivo_zip\Empresas0.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas1.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas2.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas3.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas4.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas5.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas6.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas7.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas8.zip
C:\Temp\Iesus\dados\arquivo_zip\Empresas9.zip


# 02 - Etapa de Renomeação dos Arquivos


In [4]:
for nome_arquivo in os.listdir(fr"C:\Temp\Iesus\dados\arquivo_csv"):
    
    nome_sem_extensao, _ = os.path.splitext(nome_arquivo)    
    
    nome_arquivo = fr"C:\Temp\Iesus\dados\arquivo_csv" + '\\' +nome_arquivo
    novo_nome = fr"C:\Temp\Iesus\dados\arquivo_csv" + '\\' +nome_sem_extensao + ".CSV"
    os.rename(nome_arquivo, novo_nome)


# 03 - Etapa de Ingestão dos Arquivos


In [5]:
CNPJSchema = StructType([
    StructField("CNPJ", StringType(), True),
    StructField("RAZAO_SOCIAL", StringType(), True),
    StructField("NATUREZA_JURIDICA", StringType(), True),
    StructField("QUALIFICACAO_RESPONSAVEL", StringType(), True),
    StructField("CAPITAL_SOCIAL", DecimalType(), True),
    StructField("PORTE_EMPRESA", StringType(), True),
    StructField("ENTE_FEDERATIVO", StringType(), True)
])

df_CNPJ = (spark.read.options(header='False'
                            ,delimiter=';'
                                    ,inferSchema='True'
                                    )
                                    .schema(CNPJSchema)
                                    .csv(fr'C:\Temp\Iesus\dados\arquivo_csv\*.CSV'))

In [14]:
spark.sql("""SELECT COUNT(1) total FROM {df}""", df=df_CNPJ).show()

+--------+
|   total|
+--------+
|56195990|
+--------+



# 04 - Etapa de Persistência do DataFrame em Parquet


In [15]:
caminho_do_diretorio = "C:\Temp\Iesus\dados\parquet"

if os.path.exists(caminho_do_diretorio):
    
    if not os.listdir(caminho_do_diretorio):
        
        os.rmdir(caminho_do_diretorio)        
        
    else:

        for arquivo in os.listdir(caminho_do_diretorio):
                
            arquivo_excluir = caminho_do_diretorio + '\\' + arquivo
            os.remove(arquivo_excluir)
            
        os.rmdir(caminho_do_diretorio)        
        

In [17]:

df_CNPJ.coalesce(1).write.parquet("C:\Temp\Iesus\dados\parquet")


# 04.1 - Etapa de Importação do Arquivo no Pandas


In [18]:

df_pandas = pd.read_parquet("C:\Temp\Iesus\dados\parquet")


In [19]:
df_pandas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 56195990 entries, 0 to 56195989
Data columns (total 7 columns):
 #   Column                    Dtype 
---  ------                    ----- 
 0   CNPJ                      object
 1   RAZAO_SOCIAL              object
 2   NATUREZA_JURIDICA         object
 3   QUALIFICACAO_RESPONSAVEL  object
 4   CAPITAL_SOCIAL            object
 5   PORTE_EMPRESA             object
 6   ENTE_FEDERATIVO           object
dtypes: object(7)
memory usage: 2.9+ GB


# 04.2 - Etapa de Salvar os Dados em um Arquivo CSV


In [20]:
df_pandas.to_csv(fr"C:\Temp\Iesus\dados\arquivo_final\arquivo_completo_parquet_cnpj.csv"
                 , sep=';'
                 , index=False)

# 05 - Etapa de Persistência do DataFrame em CSV


In [16]:
caminho_do_diretorio = "C:\Temp\Iesus\dados\csv"

if os.path.exists(caminho_do_diretorio):
    
    if not os.listdir(caminho_do_diretorio):
        
        os.rmdir(caminho_do_diretorio)        
        
    else:

        for arquivo in os.listdir(caminho_do_diretorio):
                
            arquivo_excluir = caminho_do_diretorio + '\\' + arquivo
            os.remove(arquivo_excluir)
            
        os.rmdir(caminho_do_diretorio)        
        

In [ ]:
df_CNPJ.coalesce(1).write.format('com.databricks.spark.csv').save('C:\Temp\Iesus\dados\csv')

In [ ]:
for nome_arquivo in os.listdir(fr"C:\Temp\Iesus\dados\csv"):
    
    if nome_arquivo.endswith("csv"):
        
        print(nome_arquivo)
    
        nome_sem_extensao, _ = os.path.splitext(nome_arquivo)    

        nome_arquivo = fr"C:\Temp\Iesus\dados\csv" + '\\' +nome_arquivo
        novo_nome = fr"C:\Temp\Iesus\dados\arquivo_final" + '\\arquivo_completo_parquet_cnpj.csv'
        os.rename(nome_arquivo, novo_nome)